In [6]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [7]:
import os
from os import listdir
import gdown
from zipfile import ZipFile
import pickle
import cv2
from os import listdir

In [8]:
from tensorflow.keras.utils import img_to_array
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            img_gan = cv2.resize(image, (32,32))
            return img_to_array(img_gan)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None
    

In [46]:

def load_dataset(directory_root):
#     directory_root = 'DCGAN_plant'
    image_list_GAN, label_list = [], []
    try:
        print("[INFO] Loading images ...")
        root_dir = listdir(directory_root)
        #print(root_dir)
        for plant_folder in root_dir :
            # remove .DS_Store from list
            if plant_folder == ".DS_Store" :
                root_dir.remove(directory)


        for plant_folder in root_dir :
    #         print("on plant folder")
            plant_image_list = listdir(f"{directory_root}/{plant_folder}")



            for single_image in plant_image_list :

    #             print("in plant folder")
                if single_image == ".DS_Store" :
                    plant_image_list.remove(single_image)

            for image in plant_image_list[:1000]:
    #             print("getting img path")
                image_directory = f"{directory_root}/{plant_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True or image_directory.endswith(".png") == True or image_directory.endswith(".PNG") == True:
    #                 print("checking if img correct format")
                    gan_img = convert_image_to_array(image_directory)
                    image_list_GAN.append(gan_img)
                    label_list.append(plant_folder)

        print("[INFO] Image loading completed")  
        return image_list_GAN, label_list
    except Exception as e:
        print(f"Error : {e}")

#     try:
#         print("[INFO] Loading images ...")
#         root_dir = listdir(directory_root)
#         for directory in root_dir :
#             # remove .DS_Store from list
#             if directory == ".DS_Store" :
#                 root_dir.remove(directory)

#         for plant_folder in root_dir :
#             plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")

#             for disease_folder in plant_disease_folder_list :
#                 # remove .DS_Store from list
#                 if disease_folder == ".DS_Store" :
#                     plant_disease_folder_list.remove(disease_folder)

#             for plant_disease_folder in plant_disease_folder_list:
#                 print(f"[INFO] Processing {plant_disease_folder} ...")
#                 plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")

#                 for single_plant_disease_image in plant_disease_image_list :
#                     if single_plant_disease_image == ".DS_Store" :
#                         plant_disease_image_list.remove(single_plant_disease_image)

#                 for image in plant_disease_image_list[:1000]:
#                     image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
#                     if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True or image_directory.endswith(".png") == True or image_directory.endswith(".PNG") == True:
#                         image_list_GAN.append(convert_image_to_array(image_directory))
#                         label_list.append(plant_disease_folder)
#         print("[INFO] Image loading completed")  
#         return image_list_GAN, label_list
#     except Exception as e:
#         print(f"Error : {e}")

In [134]:
orignal_img_list, orignal_label_list = load_dataset('Enhanced_tomato_WGAN')
print(len(orignal_img_list))

[INFO] Loading images ...
[INFO] Image loading completed
2000


In [135]:
# GAN_image_list, GAN_label_list = load_dataset('DCGAN_plant')

In [136]:
# normalized_image_list_GAN = np.array(GAN_image_list, dtype = np.float16) / 255.0
normalized_image_list_orignal = np.array(orignal_img_list, dtype = np.float16) / 255.0

In [137]:
from sklearn.preprocessing import LabelBinarizer
label_gen = LabelBinarizer()
image_labels = label_gen.fit_transform(orignal_label_list)
pickle.dump(label_gen,open('label_transform.pkl', 'wb'))
n_classes = len(label_gen.classes_)
print(n_classes)
print(image_labels)

2
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]


In [138]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# label_encoder = LabelEncoder()
# image_labels_GAN = label_encoder.fit_transform(GAN_label_list)
# n_classes = len(label_encoder.classes_)
# print(n_classes)
# print(image_labels_GAN)

# # Convert the labels to one-hot encoded vectors
# image_labels_GAN = to_categorical(image_labels_GAN, num_classes=n_classes)
# print(image_labels_GAN[3000])

In [139]:
# label_encoder = LabelEncoder()
# image_labels_orignal = label_encoder.fit_transform(orignal_label_list)
# n_classes = len(label_encoder.classes_)
# print(n_classes)
# print(image_labels_orignal)

# # Convert the labels to one-hot encoded vectors
# image_labels_orignal = to_categorical(image_labels_orignal, num_classes=n_classes)
# print(image_labels_orignal)

In [140]:
from sklearn.model_selection import train_test_split
print("making the training and testing split with 30% of the dataset as testing set")
# GAN_x_train, GAN_x_test, GAN_y_train, GAN_y_test = train_test_split(normalized_image_list_GAN, image_labels_GAN, test_size=0.3, shuffle=True,random_state = 42)

orignal_x_train, orignal_x_test, orignal_y_train, orignal_y_test = train_test_split(normalized_image_list_orignal, image_labels, test_size=0.3, shuffle=True,random_state = 42)
print(orignal_x_train.shape)

making the training and testing split with 30% of the dataset as testing set
(1400, 32, 32, 3)


In [141]:
# Define the input shape of the images
input_shape = (32, 32, 3)

# Initialize the model
model = Sequential()

# Add the first convolutional layer
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))

# Add the max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add the second convolutional layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

# Add the max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add the flatten layer
model.add(Flatten())

# Add the dense layer
model.add(Dense(128, activation='relu'))

# Add the output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the summary of the model architecture
model.summary()





Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 2304)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 128)               295040    
_________________________________________________________________
dense_21 (Dense)             (None, 1)               

In [142]:
# Train the model
model.fit(orignal_x_train, orignal_y_train, epochs=10, batch_size=16)


Epoch 1/10
88/88 [==============================] - 1s 6ms/step - loss: 0.2934 - accuracy: 0.8786
Epoch 2/10
88/88 [==============================] - 0s 5ms/step - loss: 0.0924 - accuracy: 0.9743
Epoch 3/10
88/88 [==============================] - 0s 5ms/step - loss: 0.0790 - accuracy: 0.9721
Epoch 4/10
88/88 [==============================] - 0s 5ms/step - loss: 0.0243 - accuracy: 0.9943
Epoch 5/10
88/88 [==============================] - 0s 5ms/step - loss: 0.0121 - accuracy: 0.9964
Epoch 6/10
88/88 [==============================] - 0s 5ms/step - loss: 0.0040 - accuracy: 0.9993
Epoch 7/10
88/88 [==============================] - 0s 5ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 8/10
88/88 [==============================] - 0s 5ms/step - loss: 0.0050 - accuracy: 0.9986
Epoch 9/10
88/88 [==============================] - 0s 5ms/step - loss: 8.8914e-04 - accuracy: 1.0000
Epoch 10/10
88/88 [==============================] - 0s 5ms/step - loss: 3.9978e-04 - accuracy: 1.0000


In [143]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

y_pred = model.predict(orignal_x_test)

y_pred_binary = np.where(y_pred > 0.5, 1, 0)
# y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate the true labels for the test set
y_true = orignal_y_test

accuracy = accuracy_score(orignal_y_test, y_pred_binary)
precision = precision_score(orignal_y_test, y_pred_binary, average='macro')
recall = recall_score(orignal_y_test, y_pred_binary, average='macro')
f1 = f1_score(orignal_y_test, y_pred_binary, average='macro')
# conf_matrix = confusion_matrix(y_test, y_pred_binary)


print("Accuracy:", accuracy * 100)
print("Precision:", precision * 100)
print("Recall:", recall * 100)
print("F1-score:", f1 * 100)

Accuracy: 100.0
Precision: 100.0
Recall: 100.0
F1-score: 100.0


In [132]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import label_binarize
from sklearn.metrics import multilabel_confusion_matrix






In [133]:
model.save('Enhanced_tomato_WGAN.h5')